In [2]:
import numpy as np 
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt


In [3]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
import os

# Asignar la ruta donde deseas guardar las imágenes
PROJECT_ROOT_DIR = r"C:\Users\ezequ\OneDrive\Documentos\Facultad\Optimizacion\tp1"
CHAPTER_ID = "imagenes"  # El nombre de la capeta donde voy a guardar las imagenes

# Crear la ruta completa donde se guardarán las imágenes
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)



In [6]:
path = r'C:\Users\ezequ\OneDrive\Documentos\Facultad\Aprendizaje Automatico\material-main\tp\01_aprendizaje_supervisado\datos\data.csv'
df = pd.read_csv(path)


In [6]:
df['target'].value_counts()

target
0    343
1    157
Name: count, dtype: int64

La relacion entre clases es 157/343 = 0.46 siendo la mayoritaria la clase representada por el 0 por lo que estamos ante un dataset desbalanceado

A priori hacer un split 400 100 para desarrollo validacion suena razonable , estamos dejando 20% de nuestro dataset para validar.

Capaz es un poco mucho ese porcentaje pero creo que eso aseguraria que los resultados obtenidos tendran un poco mas de robustez.

Dentro de desarrollo capaz podemos hacer algo como k folds con k = 4 para sacarle un poco mas el jugo a los datos.

train = 80%
validacion = 20%

In [7]:
def train_test_split(df,train_size=0.8):
    df = df.sample(frac=1, random_state=42).reset_index(drop=True)
    n = df.shape[0]
    cantNeg, cantPos = df['target'].value_counts()
    proporcion = cantPos / n

    cant_en_train = int(n*train_size)
    cant_en_validacion = n - cant_en_train
    
    res = []
    cant_positivas = int(proporcion * cant_en_validacion)
    cant_negativas = cant_en_validacion - cant_positivas

    contador_positivos, contador_negativos = 0, 0
    for index, value in enumerate(df['target']):
        if value == 0:
            contador_negativos += 1
            res.append(index)
        
        if contador_negativos == cant_negativas:
            break
    for index, value in enumerate(df['target']):

        if value == 1:
            contador_positivos += 1
            res.append(index)
        
        if contador_positivos == cant_positivas:
            break
    
    df_dev = df.loc[~df.index.isin(res)].reset_index(drop=True)
    df_val = df.loc[res].reset_index(drop=True)
    return df_dev,df_val

In [8]:
df_dev , df_val =  train_test_split(df)


# Ejercicio 2

In [52]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score, roc_auc_score , average_precision_score
from sklearn.model_selection import cross_val_predict


In [10]:
X_train = df_dev.drop(columns=['target']).values

y_train = df_dev['target'].values

X_val   = df_val.drop(columns=['target']).values

y_val   =  df_val['target'].values

In [11]:
tree_clf = DecisionTreeClassifier(max_depth=3, random_state=51)
tree_clf.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=3, random_state=51)

In [54]:

scoring = ['accuracy', 'roc_auc', 'average_precision']

skf = StratifiedKFold(n_splits=5)

cv_results = cross_validate(tree_clf, X_train, y_train, cv = skf, scoring = scoring,return_train_score = True)

predicted_labels = cross_val_predict(tree_clf, X_train, y_train, cv = skf)

metricas_globales = {'Accuracy Global' : accuracy_score(y_train, predicted_labels),
                     'Auc Global' : roc_auc_score(y_train, predicted_labels),
                      'Average Precision Global' : average_precision_score(y_train, predicted_labels)
}

cv_results.update(metricas_globales)

df_resultados_ej_2_2 = pd.DataFrame({key: cv_results[key] for key in list(cv_results.keys())[3:]})

df_resultados_ej_2_2.index = df_resultados_ej_2_2.index + 1 # asi el indice del df se correponde con el k


In [65]:
df_resultados_ej_2_2

,test_accuracy,train_accuracy,test_roc_auc,train_roc_auc,test_average_precision,train_average_precision,Accuracy Global,Auc Global,Average Precision Global
1,0.6500,0.834375,0.585470,0.770614,0.432637,0.656625,0.695,0.599467,0.386459
2,0.7000,0.818750,0.732000,0.852706,0.484188,0.713912,0.695,0.599467,0.386459
3,0.6750,0.825000,0.613455,0.796826,0.404476,0.657074,0.695,0.599467,0.386459
4,0.6875,0.809375,0.596000,0.854492,0.394547,0.713593,0.695,0.599467,0.386459
5,0.7625,0.837500,0.705455,0.836928,0.516564,0.733381,0.695,0.599467,0.386459


In [69]:
cv_results.keys()

dict_keys(['fit_time', 'score_time', 'indices', 'test_accuracy', 'train_accuracy', 'test_roc_auc', 'train_roc_auc', 'test_average_precision', 'train_average_precision', 'Accuracy Global', 'Auc Global', 'Average Precision Global'])

Basado en pagina 288 de Introduction to Machine Learning with Python

In [72]:
alturas = [3, 5, None]
criterios = ["gini", "entropy"]

resultados = {}

for altura in alturas:
    for criterio in criterios:
        
        tree = DecisionTreeClassifier(max_depth = altura, criterion = criterio, random_state = 51)
                
        cv_results_grid = cross_validate(tree, X_train, y_train, cv = skf, scoring = 'accuracy', return_train_score = True)
        
        accuracy_train = cv_results_grid['train_score'].mean()

        accuracy_test =  cv_results_grid['test_score'].mean()

        
        
        resultados[(altura, criterio)] = (accuracy_train,accuracy_test)

results_df = pd.DataFrame(resultados.values(), index=resultados.keys(), columns=['Train Score', 'Test Score'])



In [61]:
results_df

Train Score  Test Score
3   gini        0.661111    0.633333
    entropy     0.638889    0.755556
5   gini        0.644444    0.600000
    entropy     0.650000    0.700000
NaN gini        0.630556    0.577778
    entropy     0.652778    0.700000

# Ejercicio 3

In [64]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

# Define the hyperparameter grid
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [3, 5, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    #'max_features': ['auto', 'sqrt', 'log2', None]
}

# Create the Decision Tree classifier
tree = DecisionTreeClassifier()

# Create the RandomizedSearchCV object
random_search = RandomizedSearchCV(estimator=tree, param_distributions=param_grid, scoring='roc_auc', cv=5)

# Fit the RandomizedSearchCV object to the data
random_search.fit(X_train, y_train)

# Get the best model and its performance
best_model = random_search.best_estimator_
best_score = random_search.best_score_

# Print the best model and its performance
print("Best Model:", best_model)
print("Best AUCROC Score:", best_score)

Best Model: DecisionTreeClassifier(min_samples_split=5)
Best AUCROC Score: 0.636304347826087


In [66]:
from sklearn.neighbors import KNeighborsClassifier

# Create KNN classifier
knn = KNeighborsClassifier(n_neighbors=3)

# Train the model using the development data
knn.fit(X_train, y_train)

# Predict the target variable for the test data
y_pred_knn = knn.predict(X_test)

# Calculate the accuracy score for KNN
accuracy_knn = accuracy_score(y_test, y_pred_knn)

# Compare the accuracy scores of Decision Tree and KNN
comparison = {'Decision Tree': accuracy_test, 'KNN': accuracy_knn}
comparison


{'Decision Tree': 0.7, 'KNN': 0.8}